In [ ]:
#!/usr/bin/env python
# coding: utf-8

from __future__ import division

import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, CSVLogger
import tensorflow as tf
import time
import os

from tensorflow.keras.layers import Input, Dense, ReLU, Softmax, Dropout, Conv2D, MaxPool2D, Flatten, Reshape
from tensorflow.keras import Model

# %matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def tile_shuffle(images, tile_size=None):
    if tile_size is None:
        tile_size = np.random.choice((2, 4, 7, 14), size=1)[0]
    n_tiles = 28//tile_size # in 1-D
    
    n_im = images.shape[0]
    
    images = np.reshape(images, (n_im, tile_size, n_tiles, tile_size, n_tiles), order='F')
    images = np.transpose(images, (0, 2, 4, 3, 1))
    images = np.reshape(images, (n_im, n_tiles*n_tiles, tile_size, tile_size), order='F')
    
    images = images[:, np.random.permutation(n_tiles*n_tiles)]
    
    images = np.reshape(images, (n_im, n_tiles, n_tiles, tile_size, tile_size), order='F')
    images = np.transpose(images, (0, 3, 2, 4, 1))
    images = np.reshape(images, (n_im, n_tiles*tile_size, n_tiles*tile_size), order='F')
    images = np.reshape(images, (n_im, 784))
    
    return images

In [ ]:
def mnist_generator(dir_mnist, batch_size=16, dataset='train', random_order=True,
                    null_types=None, p_null_class=1/11, tile_size=None):
    # dataset: May be 'train' or 'test'
    # null_types: A string of character, with each character indicating a type of
    #             image that will be used for the null class:
    #                 'u' - uniform noise
    #                 's' - tile-shuffled MNIST images
    #                 'm' - mixed: two MNIST images (of different classes) averaged/mixed together
    #             For example: 'um' indicates that null samples will include both uniform noise
    #             samples and mixed sampled.
            
    from mnist import MNIST
    mndata = MNIST(dir_mnist)

    if dataset=='train':
        images, labels = mndata.load_training()
    elif dataset=='test':
        images, labels = mndata.load_testing()
    else:
        raise Exception("MNIST dataset must be 'train' or 'test'")

    images = np.array(images).astype(np.float32) / 255
    labels = np.array(labels).astype(np.float32)
    n_samples = labels.shape[0]
    
    global idx_pointer
    idx_pointer = 0
    def get_next_sample_indices(n_wanted, random_order=random_order):
        global idx_pointer
        if random_order:
            ix_mnist = np.random.choice(n_samples, size=n_wanted, replace=False)
        else:
            ix_mnist = np.arange(idx_pointer, idx_pointer+n_wanted, dtype=np.int)
            ix_mnist = ix_mnist % n_samples
            idx_pointer += n_wanted
            if idx_pointer >= n_samples:
                idx_pointer = idx_pointer % n_samples    
        return ix_mnist
        
    while True:
        # Determine how many samples to get of each type/class
        if null_types is None or null_types=='':
            n_mnist = batch_size
            n_uniform = 0
            n_shuffled = 0
            n_mixed = 0            
        else:
            # Warning: not checking to make sure null_types string is legitimate
            prob_type = np.array(['u' in null_types, 's' in null_types, 'm' in null_types])
            prob_type = prob_type / np.sum(prob_type)
            
            n_nulls = np.sum(np.random.uniform(size=(batch_size)) < p_null_class)            
            n_mnist = batch_size - n_nulls

            iid_type = np.random.choice([0, 1, 2], size=n_nulls, replace=True, p=prob_type)
            n_uniform = np.sum(iid_type==0)
            n_shuffled = np.sum(iid_type==1)
            n_mixed = np.sum(iid_type==2)

        # Randomly select mnist samples.
        ix_mnist = get_next_sample_indices(n_mnist)
        labels_batch = labels[ix_mnist]
        images_batch = images[ix_mnist]

        # Generate the uniform noise null images.
        labels_other = np.full((n_uniform), 10, dtype=np.float32)
        images_other = np.random.uniform(0, high=1, size=(n_uniform, 784)).astype(np.float32)
        labels_batch = np.concatenate((labels_batch, labels_other))
        images_batch = np.concatenate((images_batch, images_other))
        
        # Generate the shuffled null images.
        # All images of batch have pixels shuffled in same order.
        labels_other = np.full((n_shuffled), 10, dtype=np.float32)
        ix_mnist = get_next_sample_indices(n_shuffled)
        images_other = tile_shuffle(images[ix_mnist], tile_size=tile_size)
        labels_batch = np.concatenate((labels_batch, labels_other))
        images_batch = np.concatenate((images_batch, images_other))

        # Generate the mixed null images.
        labels_other = np.full((n_mixed), 10, dtype=np.float32)
        images_other = np.empty((0, 784), dtype=np.float32)
        cnt = 0
        while cnt < n_mixed:
            got_one = False
            while not got_one:
                ix_mnist = get_next_sample_indices(2)
                if labels[ix_mnist[0]] != labels[ix_mnist[1]]:
                    got_one = True
                    im = ((images[ix_mnist[0]] + images[ix_mnist[1]]) / 2).astype(np.float32)
                    images_other = np.concatenate((images_other, np.expand_dims(im, axis=0)))
            cnt += 1
        labels_batch = np.concatenate((labels_batch, labels_other))
        images_batch = np.concatenate((images_batch, images_other))
        
        # yield images_batch, labels_batch
        yield images_batch, labels_batch, [None]
        # Added [None] to get rid of mystery error messsage in TF 2.1:
        #    sample_weight modes were coerced from ... to ['...']

In [ ]:
def ramp_generator(batch_size=16, null_types=None, p_null_class=1/3):
    # null_types: A string of character, with each character indicating a type of
    #             image that will be used for the null class:
    #                 'u' - uniform noise
    #                 's' - shuffled samples
    #             For example: 'us' indicates that null samples will include both uniform noise
    #             samples and shuffled sampled.

    proto_ramps = np.array([[0.666, 0.333], [0.333, 0.666]])
    
    def create_ramps(n_samples):
        ramps = np.zeros((n_samples, 3), dtype=np.float32)
        
        n0 = np.sum(np.random.uniform(size=(n_samples)) < 0.5)
        n1 = n_samples - n0
                
        # Probably some faster/smarter way of doing this
        shifts = np.random.choice(2, size=(n0))
        for i in range(n0):
            ramps[i, shifts[i]:shifts[i]+2] = proto_ramps[0]
        shifts = np.random.choice(2, size=(n1))
        for i in range(n1):
            ramps[i+n0, shifts[i]:shifts[i]+2] = proto_ramps[1]
        
        # Add some noise
        ramps += np.random.uniform(-0.05, high=0.05, size=(n_samples, 3)).astype(np.float32)
        ramps = np.clip(ramps, 0, 1)
        
        labels = np.zeros((n_samples,1))
        labels[n0:,0] = 1
        
        return ramps, labels
    
    while True:
        # Determine how many samples to get of each type/class
        if null_types is None or null_types=='':
            n_ramps = batch_size
            n_uniform = 0
            n_shuffled = 0
        else:
            # Warning: not checking to make sure null_types string is legitimate
            prob_type = np.array(['u' in null_types, 's' in null_types])
            prob_type = prob_type / np.sum(prob_type)
            
            n_nulls = np.sum(np.random.uniform(size=(batch_size)) < p_null_class)            
            n_ramps = batch_size - n_nulls

            iid_type = np.random.choice([0, 1], size=n_nulls, replace=True, p=prob_type)
            n_uniform = np.sum(iid_type==0)
            n_shuffled = np.sum(iid_type==1)

        # Get true samples.
        images_batch, labels_batch = create_ramps(n_ramps)

        # Generate the uniform noise null images.
        labels_other = np.full((n_uniform, 1), 2, dtype=np.float32)
        images_other = np.random.uniform(0, high=1, size=(n_uniform, 3)).astype(np.float32)
        labels_batch = np.concatenate((labels_batch, labels_other))
        images_batch = np.concatenate((images_batch, images_other))
        
        # Generate the shuffled null images.
        # All images of batch have pixels shuffled in same order.
        labels_other = np.full((n_shuffled, 1), 2, dtype=np.float32)
        images_other, _ = create_ramps(n_shuffled)
        ix = np.random.permutation(3)
        images_other = images_other[:,ix]
        labels_batch = np.concatenate((labels_batch, labels_other))
        images_batch = np.concatenate((images_batch, images_other))
        
        yield images_batch, labels_batch